# Regrid beam maps
to match continuum image sizes and HI image sizes for DR1


In [1]:
import os
import sys
import numpy as np
import matplotlib as mpl
from astropy.io import fits as fits_astro
import apercal.libs.lib as lib


fits = lib.miriad('fits')
regrid = lib.miriad('regrid')
    

# Continuum 

In [2]:
# uese this version to regrid old beam maps
# this version flips the beam maps

def change_centre_flip(image_dir, beam_dir, out_dir, image, beam):
    """
    This function changes the centre of the beam model to the same as the HI cube

    work_dir: directory with the HI image, this is also where all the new files will be saved 
    beam_dir: directory with the beam model
    image: HI data cube
    beam: beam model file (40x40 pixel)
    """
    print('Recentreing beam map.')
    hdulist = fits_astro.open('{}{}'.format(image_dir, image))
    hdulist_beam = fits_astro.open('{}{}'.format(beam_dir, beam))
    d = hdulist[0].data
    h = hdulist[0].header
    hdulist_beam[0].data = np.flipud(hdulist_beam[0].data)
    h_beam = hdulist_beam[0].header
    hdulist_beam[0].header['CRVAL1'] = hdulist[0].header['CRVAL1']
    hdulist_beam[0].header['CRVAL2'] = hdulist[0].header['CRVAL2']
    hdulist_beam.writeto('{}{}_rec.fits'.format(out_dir, beam[:-5]), overwrite=True)

In [3]:
def change_centre(image_dir, beam_dir, out_dir, image, beam):
    """
    This function changes the centre of the beam model to the same as the HI cube

    work_dir: directory with the HI image, this is also where all the new files will be saved 
    beam_dir: directory with the beam model
    image: HI data cube
    beam: beam model file (40x40 pixel)
    """
    print('Recentreing beam map.')
    hdulist = fits_astro.open('{}{}'.format(image_dir, image))
    hdulist_beam = fits_astro.open('{}{}'.format(beam_dir, beam))
    d = hdulist[0].data
    h = hdulist[0].header
    h_beam = hdulist_beam[0].header
    hdulist_beam[0].header['CRVAL1'] = hdulist[0].header['CRVAL1']
    hdulist_beam[0].header['CRVAL2'] = hdulist[0].header['CRVAL2']
    hdulist_beam.writeto('{}{}_rec.fits'.format(out_dir, beam[:-5]), overwrite=True)

In [4]:
def change_header(beam_dir, out_dir, beam_2, beam):
    """
    This function adds the frequency information to the header

    work_dir: directory with the HI image, this is also where all the new files will be saved 
    beam_dir: directory with the beam model
    image: HI data cube
    beam: beam model file (40x40 pixel)
    """
    print('Recentreing beam map.')
    hdulist = fits_astro.open('{}{}'.format(beam_dir, beam))
    hdulist_beam = fits_astro.open('{}{}'.format(out_dir, beam_2))
    hdulist_beam[0].header['CRPIX3'] = hdulist[0].header['CRPIX3']
    hdulist_beam[0].header['CDELT3'] = hdulist[0].header['CDELT3']
    hdulist_beam[0].header['CUNIT3'] = hdulist[0].header['CUNIT3']
    hdulist_beam[0].header['CTYPE3'] = 'FREQ'
    hdulist_beam[0].header['CRVAL3'] = hdulist[0].header['CRVAL3']
    hdulist_beam[0].header['NAXIS'] = 3
    hdulist_beam[0].header['NAXIS3'] = 1
    hdulist_beam.writeto('{}{}'.format(out_dir, beam_2), overwrite=True) 

In [5]:
# set beam centre to 0, 0

def change_centre_v2(out_dir, beam):
    """
    This function changes the centre of the beam model to the same as the HI cube

    work_dir: directory with the HI image, this is also where all the new files will be saved 
    beam_dir: directory with the beam model
    image: HI data cube
    beam: beam model file (40x40 pixel)
    """
    print('Recentreing beam map.')
    hdulist_beam = fits_astro.open('{}{}'.format(out_dir, beam))
    hdulist_beam[0].header['CRVAL1'] = 0.0
    hdulist_beam[0].header['CRVAL2'] = 0.0
    hdulist_beam.writeto('{}{}'.format(out_dir, beam), overwrite=True)

In [6]:
def regrid_in_miriad(image_dir, work_dir, image, beam):
    """
    Regrid the beam image in miriad to the HI image

    work_dir: directory with the HI image, this is also where all the new files will be saved 
    image: HI data cube
    beam: beam model file (40x40 pixel)
    """
    print('Regridding im miriad')
    
    fits = lib.miriad('fits')
    regrid = lib.miriad('regrid')

    # convert files to miriad:

    fits.in_ = '{}{}'.format(image_dir, image)
    fits.out = '{}{}.mir'.format(work_dir, image[:-5])
    fits.op = 'xyin'
    fits.go()

    fits.in_ = '{}{}'.format(work_dir, beam)
    fits.out = '{}{}.mir'.format(work_dir, beam[:-5])
    fits.op = 'xyin'
    fits.go()

    # regrid beam image

    regrid.in_ = '{}{}.mir'.format(work_dir, beam[:-5])
    regrid.out = '{}{}_reg.mir'.format(work_dir, beam[:-9])
    regrid.tin = '{}{}.mir'.format(work_dir, image[:-5])
    regrid.axes = '1,2'
    regrid.go()

    # convert regrided beam image to fits

    fits.in_ = '{}{}_reg.mir'.format(work_dir, beam[:-9])
    fits.out = '{}{}_reg.fits'.format(work_dir, beam[:-9])
    fits.op = 'xyout'
    fits.go()
    

In [7]:
# use this for the new beams

taskid = '201028'

for beams in range(0,40):
    beam_dir = '/tank/apertif/driftscans/fits_files/{}/beam_models/chann_5/'.format(taskid)
    image_dir = '/tank/denes/continuum_images/'
    out_dir = '/tank/denes/drift_scan/{}/cont/'.format(taskid)
    
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)

    image = 'continuum_b{:02}.fits'.format(beams)
    beam = '{}_{:02}_I_model.fits'.format(taskid, beams)
    beam_2 = '{}_{:02}_I_model_rec.fits'.format(taskid, beams)

    change_centre(image_dir, beam_dir, out_dir, image, beam)
    regrid_in_miriad(image_dir, out_dir, image, beam_2)
    
    os.popen('rm -rf {}*.mir'.format(out_dir))


Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Regridding im miriad
Recentreing

In [8]:
# run this for the old beams

date = '200430'

for beams in range(0,40):
    beam_dir = '/tank/apertif/driftscans/fits_files/{}/beam_models_old/chann_5/'.format(date)
    image_dir = '/tank/denes/continuum_images/'
    out_dir = '/tank/denes/drift_scan/{}/cont/'.format(date)
    
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)

    image = 'continuum_b04.fits'
    beam = '{}_{:02}_I_model.fits'.format(date, beams)
    beam_2 = '{}_{:02}_I_model_rec.fits'.format(date, beams)
    beam_3 = '{}_{:02}_I_model_reg.fits'.format(date, beams)

    change_centre_flip(image_dir, beam_dir, out_dir, image, beam)
    regrid_in_miriad(image_dir, out_dir, image, beam_2)
    change_header(beam_dir, out_dir, beam_3, beam)
    
    os.popen('rm -rf {}*.mir'.format(out_dir))
    os.popen('rm -rf {}*_rec.fits'.format(out_dir))

Recentreing beam map.
Regridding im miriad
Recentreing beam map.


Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Recentreing beam map.
Reg

# HI cubes

In [113]:
# uese this version to regrid old beam maps
# this version flips the beam maps

def change_centre_flip_hi(image_dir,beam_dir, out_dir, image, beam):
    """
    This function changes the centre of the beam model to the same as the HI cube

    work_dir: directory with the HI image, this is also where all the new files will be saved 
    beam_dir: directory with the beam model
    image: HI data cube
    beam: beam model file (40x40 pixel)
    """
    print('Recentreing beam map.')
    hdulist = fits_astro.open('{}{}'.format(image_dir, image))
    hdulist_beam = fits_astro.open('{}{}'.format(beam_dir, beam))
    d = hdulist[0].data
    h = hdulist[0].header
    hdulist_beam[0].data = np.flipud(hdulist_beam[0].data)
    h_beam = hdulist_beam[0].header
    hdulist_beam[0].header['CRVAL1'] = hdulist[0].header['CRVAL1']
    hdulist_beam[0].header['CRVAL2'] = hdulist[0].header['CRVAL2']
    hdulist_beam.writeto('{}{}_rec.fits'.format(out_dir, beam[:-5]), overwrite=True)

In [114]:
def regrid_in_miriad_hi(image_dir, work_dir, image, beam):
    """
    Regrid the beam image in miriad to the HI image

    work_dir: directory with the HI image, this is also where all the new files will be saved 
    image: HI data cube
    beam: beam model file (40x40 pixel)
    """
    print('Regridding im miriad')
    
    fits = lib.miriad('fits')
    regrid = lib.miriad('regrid')

    # convert files to miriad:

    fits.in_ = '{}{}'.format(image_dir, image)
    fits.out = '{}{}.mir'.format(work_dir, image[:-5])
    fits.op = 'xyin'
    fits.go()

    fits.in_ = '{}{}'.format(work_dir, beam)
    fits.out = '{}{}.mir'.format(work_dir, beam[:-5])
    fits.op = 'xyin'
    fits.go()

    # regrid beam image

    regrid.in_ = '{}{}.mir'.format(work_dir, beam[:-5])
    regrid.out = '{}{}_reg.mir'.format(work_dir, beam[:-9])
    regrid.tin = '{}{}.mir'.format(work_dir, image[:-5])
    regrid.axes = '1,2'
    regrid.go()

    # convert regrided beam image to fits

    fits.in_ = '{}{}_reg.mir'.format(work_dir, beam[:-9])
    fits.out = '{}{}_reg.fits'.format(work_dir, beam[:-9])
    fits.op = 'xyout'
    fits.go()
    

In [160]:
# run this for the old beams

date = '200430'
cubes = ['cube0', 'cube1', 'cube2']
maps = ['chann_4', 'chann_6', 'chann_9']

for i in range(len(maps)):
    for beams in range(0,40):
        beam_dir = '/tank/apertif/driftscans/fits_files/{}/beam_models_old/{}/'.format(date, maps[i])
        image_dir = '/tank/denes/drift_scan/'
        out_dir = '/tank/denes/drift_scan/{}/{}/'.format(date,cubes[i])

        if not os.path.exists(out_dir):
            os.mkdir(out_dir)

        image = 'HI_mom0.fits'
        beam = '{}_{:02}_I_model.fits'.format(date, beams)
        beam_2 = '{}_{:02}_I_model_rec.fits'.format(date, beams)
        beam_3 = '{}_{:02}_I_model_reg.fits'.format(date, beams)

        change_centre_flip_hi(image_dir,beam_dir, out_dir, image, beam)
        regrid_in_miriad_hi(image_dir, out_dir, image, beam_2)
        change_header(beam_dir, out_dir, beam_3, beam)

        os.popen('rm -rf {}*.mir'.format(out_dir))
        os.popen('rm -rf {}*_rec.fits'.format(out_dir))

Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Recentreing beam map.
Regridding im miriad
Recentreing beam map.
Recentreing beam map.
Reg

In [12]:


#hdulist = fits_astro.open('/tank/denes/continuum_images/continuum_b01.fits')
hdulist = fits_astro.open('/tank/apertif/driftscans/fits_files/190912/beam_models_old/chann_3/190912_01_I_model.fits')
#hdulist_beam = fits_astro.open('{}{}'.format(beam_dir, beam))
d = hdulist[0].data
h = hdulist[0].header

print(h['CRVAL3']) #CTYPE3
print(h['*3'])

1289799250.0
CRPIX3  =                  1.0 / Pixel coordinate of reference point            CDELT3  =           18310500.0 / [Hz] Coordinate increment at reference point   CUNIT3  = 'Hz'                 / Units of coordinate increment and value        CTYPE3  = 'FREQ'               / Frequency (linear)                             CRVAL3  =         1289799250.0 / [Hz] Coordinate value at reference point       END                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        